In [24]:
import pandas as pd
import numpy as np

In [25]:
df = pd.read_csv('base_agencias_ficticia.csv', delimiter=';')
df.head()

,agencia,ti,to,indice_indisp,vol_atms,vol_transacao,grupo_controle,onda_fechamento,anomes,anomes_fechamento,periodo_fechamento
0,7165,5947,95845,0.062048,13,11519,NAO,3,202508,202508,POS
1,7047,4620,84369,0.054759,15,19406,SIM,NAO SE APLICA,202504,NAO SE APLICA,PRE
2,8551,13491,122382,0.110237,11,3719,SIM,NAO SE APLICA,202508,NAO SE APLICA,POS
3,8458,14652,102152,0.143433,16,14827,SIM,NAO SE APLICA,202506,NAO SE APLICA,PRE
4,4965,14491,102722,0.141070,7,10329,SIM,NAO SE APLICA,202505,NAO SE APLICA,PRE


In [26]:
df.dtypes

agencia                 int64
ti                      int64
to                      int64
indice_indisp         float64
vol_atms                int64
vol_transacao           int64
grupo_controle         object
onda_fechamento        object
anomes                  int64
anomes_fechamento      object
periodo_fechamento     object
dtype: object

In [27]:
nao_grupo_controle = df[df['grupo_controle'] == 'NAO']
grupo_controle = df[df['grupo_controle'] == 'SIM']

nao_grupo_controle_pre = nao_grupo_controle[nao_grupo_controle['periodo_fechamento'] == 'PRE']
grupo_controle_pre = grupo_controle[grupo_controle['periodo_fechamento'] == 'PRE']

In [28]:
nao_grupo_controle_pre.head()

,agencia,ti,to,indice_indisp,vol_atms,vol_transacao,grupo_controle,onda_fechamento,anomes,anomes_fechamento,periodo_fechamento
8,2648,7135,91736,0.077778,16,13710,NAO,2,202504,202507,PRE
11,975,9650,92078,0.104802,5,11411,NAO,3,202506,202509,PRE
27,2825,7100,88226,0.080475,7,11356,NAO,3,202508,202509,PRE
34,4271,17741,77347,0.229369,16,8412,NAO,2,202506,202507,PRE
35,2516,11692,104963,0.111392,5,11325,NAO,2,202505,202507,PRE


In [29]:
grupo_controle_pre.head()

,agencia,ti,to,indice_indisp,vol_atms,vol_transacao,grupo_controle,onda_fechamento,anomes,anomes_fechamento,periodo_fechamento
1,7047,4620,84369,0.054759,15,19406,SIM,NAO SE APLICA,202504,NAO SE APLICA,PRE
3,8458,14652,102152,0.143433,16,14827,SIM,NAO SE APLICA,202506,NAO SE APLICA,PRE
4,4965,14491,102722,0.141070,7,10329,SIM,NAO SE APLICA,202505,NAO SE APLICA,PRE
5,8237,11017,96652,0.113986,12,12056,SIM,NAO SE APLICA,202506,NAO SE APLICA,PRE
7,7448,9272,80644,0.114974,6,2016,SIM,NAO SE APLICA,202506,NAO SE APLICA,PRE


In [30]:
grupo_controle_pre_logica_anomes = grupo_controle_pre
nao_grupo_controle_pre_logica_anomes = nao_grupo_controle_pre

grupo_controle_pre_logica_anomes = (
    grupo_controle_pre.groupby('agencia')['anomes'].apply(lambda x: sorted(set(x))).reset_index()
)

nao_grupo_controle_pre_logica_anomes = (
    nao_grupo_controle_pre.groupby('agencia')['anomes'].apply(lambda x: sorted(set(x))).reset_index()
)

grupo_controle_pre_logica_anomes = grupo_controle_pre_logica_anomes.drop_duplicates(subset=['agencia'])
nao_grupo_controle_pre_logica_anomes = nao_grupo_controle_pre_logica_anomes.drop_duplicates(subset=['agencia'])
grupo_controle_pre_logica_anomes.to_csv('test/logica_anomes_validacao.csv')
grupo_controle_pre_logica_anomes

,agencia,anomes
0,1,[202506]
1,2,[202504]
2,10,[202505]
3,23,[202505]
4,28,[202506]
...,...,...
1409,9964,[202505]
1410,9979,[202506]
1411,9988,[202506]
1412,9989,[202506]


In [31]:
resultados_pre_logica = []

for index_nao, row_nao in nao_grupo_controle_pre_logica_anomes.iterrows():
    agencia_nao = row_nao['agencia']
    anomes_nao = row_nao['anomes']

    grupo_controle_filtrado = grupo_controle_pre_logica_anomes[
        grupo_controle_pre_logica_anomes['anomes'].apply(lambda x: x[0] == anomes_nao[0])
    ]

    if grupo_controle_filtrado.empty:
        continue

    grupo_controle_filtrado['tamanho_anomes'] = grupo_controle_filtrado['anomes'].apply(len)
    tamanho_minimo = len(anomes_nao)
    grupo_controle_filtrado = grupo_controle_filtrado[
        grupo_controle_filtrado['tamanho_anomes'] >= tamanho_minimo
    ]

    if grupo_controle_filtrado.empty:
        continue

    grupo_controle_filtrado = grupo_controle_filtrado.sort_values(by='tamanho_anomes', ascending=False)
    top_5_agencias = grupo_controle_filtrado.head(5)

for index_sim, row_sim in grupo_controle_filtrado.iterrows():
    resultados_pre_logica.append({
        'agencia_ref': agencia_nao,
        'anomes_agencia_ref': anomes_nao,
        'agencia_grp_controle': row_sim['agencia'],
        'anomes_grp_controle': row_sim['anomes']
    })
    
lista_ag_proximas_anomes = pd.DataFrame(resultados_pre_logica)
lista_ag_proximas_anomes.to_csv('test/lista_agencias_proximas_pre_logica.csv', index=False)

C:\Users\josea\AppData\Local\Temp\ipykernel_19164\1677683423.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_controle_filtrado['tamanho_anomes'] = grupo_controle_filtrado['anomes'].apply(len)
C:\Users\josea\AppData\Local\Temp\ipykernel_19164\1677683423.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_controle_filtrado['tamanho_anomes'] = grupo_controle_filtrado['anomes'].apply(len)
C:\Users\josea\AppData\Local\Temp\ipykernel_19164\1677683423.py:14: SettingWithCopyWarning: 
A value is tr

In [32]:
grupo_contorle_agg = grupo_controle_pre.groupby(['agencia', 'anomes']).agg({
    'ti': 'sum',
    'to': 'sum',
    'vol_atms': 'sum',
    'vol_transacao': 'sum',
}).reset_index()

nao_grupo_contorle_agg = nao_grupo_controle_pre.groupby(['agencia', 'anomes']).agg({
    'ti': 'sum',
    'to': 'sum',
    'vol_atms': 'sum',
    'vol_transacao': 'sum',
}).reset_index()


grupo_contorle_agg['indice_indisp'] = grupo_contorle_agg['ti'] / grupo_contorle_agg['to']
nao_grupo_contorle_agg['indice_indisp'] = nao_grupo_contorle_agg['ti'] / grupo_contorle_agg['to']
grupo_contorle_agg.head()

,agencia,anomes,ti,to,vol_atms,vol_transacao,indice_indisp
0,1,202506,4835,103917,13,10980,0.046528
1,2,202504,15747,195605,31,29386,0.080504
2,10,202505,15487,109151,15,19613,0.141886
3,23,202505,2713,100023,12,15869,0.027124
4,28,202506,14535,72725,5,3292,0.199862


In [33]:
grupo_contorle_agg['agencia'] = grupo_contorle_agg['agencia'].astype(int)
grupo_contorle_agg['anomes'] = grupo_contorle_agg['anomes'].astype(int)
nao_grupo_contorle_agg['agencia'] = nao_grupo_contorle_agg['agencia'].astype(int)
nao_grupo_contorle_agg['anomes'] = nao_grupo_contorle_agg['anomes'].astype(int)

In [35]:
resultados = []

def garantir_lista(val):
    if isinstance(val, (int, float)):
        return [int(val)]
    elif isinstance(val, str):
        val = val.replace('[', '').replace(']', '').replace(' ', '')
        return [int(x) for x in val.split(',') if x.strip().isdigit()]
    elif isinstance(val, list):
        return [int(x) for x in val]
    else:
        return []

# Cálculo indice de indisponibilidade
for _, row_lista in lista_ag_proximas_anomes.iterrows():
    agencia_ref = int(row_lista['agencia_ref'])
    anomes_agencia_ref = garantir_lista(row_lista['anomes_agencia_ref'])
    agencia_grp_controle = garantir_lista(row_lista['agencia_grp_controle'])
    anomes_grp_controle = garantir_lista(row_lista['anomes_grp_controle'])

    df_ref = nao_grupo_contorle_agg[
        (nao_grupo_contorle_agg['agencia'] == agencia_ref) &
        (nao_grupo_contorle_agg['anomes'].isin(anomes_agencia_ref))
    ]

    df_grp = grupo_contorle_agg[
        (grupo_contorle_agg['agencia'].isin(agencia_grp_controle)) &
        (grupo_contorle_agg['anomes'].isin(anomes_grp_controle))
    ]

    distancias_globais = []
    for ag_cand in df_grp['agencia'].unique():
        df_cand = df_grp[df_grp['agencia'] == ag_cand]
        df_merged = pd.merge(df_ref, df_cand, on='anomes', suffixes=('_ref', '_grp'))
        if df_merged.empty:
            continue
        distancia_total = np.linalg.norm(df_merged['indice_indisp_ref'] - df_merged['indice_indisp_grp'])
        distancias_globais.append({'agencia_grp_controle': ag_cand, 'distancia_total': distancia_total})

    if not distancias_globais:
        continue

    top5 = sorted(distancias_globais, key=lambda x: x['distancia_total'])[:5]

    for match in top5:
        ag_cand = match['agencia_grp_controle']
        df_cand = df_grp[df_grp['agencia'] == ag_cand]
        df_merged = pd.merge(df_ref, df_cand, on='anomes', suffixes=('_ref', '_grp'))

        for _, row in df_merged.iterrows():
            resultados.append({
                'agencia_ref': agencia_ref,
                'anomes_ref': row['anomes'],
                'agencia_grp_controle': ag_cand,
                'indice_indisp_ref': row['indice_indisp_ref'],
                'indice_indisp_grp_controle': row['indice_indisp_grp'],
                'vol_atms_ref': row['vol_atms_ref'],
                'vol_atms_grp_controle': row['vol_atms_grp'],
                'vol_transacao_ref': row['vol_transacao_ref'],
                'vol_transacao_grp_controle': row['vol_transacao_grp'],
                'distancia_euclidiana_indisp': match['distancia_total']
            })

lista_ag_proximas_indisp = pd.DataFrame(resultados)

# Cálculo volume de ATMs
resultados_vol_atms = []

for agencia_ref in lista_ag_proximas_indisp['agencia_ref'].unique():
    df_ref = lista_ag_proximas_indisp[lista_ag_proximas_indisp['agencia_ref'] == agencia_ref]

    distancias_vol = []
    for ag_cand in df_ref['agencia_grp_controle'].unique():
        df_cand = df_ref[df_ref['agencia_grp_controle'] == ag_cand]
        if df_cand.empty:
            continue

        distancia = np.linalg.norm(df_cand['vol_atms_ref'] - df_cand['vol_atms_grp_controle'])
        distancias_vol.append({'agencia_grp_controle': ag_cand, 'distancia_total_vol': distancia})

    top3 = sorted(distancias_vol, key=lambda x: x['distancia_total_vol'])[:3]

    for match in top3:
        ag_cand = match['agencia_grp_controle']
        df_cand = df_ref[df_ref['agencia_grp_controle'] == ag_cand]

        for _, row in df_cand.iterrows():
            resultados_vol_atms.append({
                'agencia_ref': agencia_ref,
                'anomes_ref': row['anomes_ref'],
                'agencia_grp_controle': ag_cand,
                'indice_indisp_ref': row['indice_indisp_ref'],
                'indice_indisp_grp_controle': row['indice_indisp_grp_controle'],
                'vol_atms_ref': row['vol_atms_ref'],
                'vol_atms_grp_controle': row['vol_atms_grp_controle'],
                'vol_transacao_ref': row['vol_transacao_ref'],
                'vol_transacao_grp_controle': row['vol_transacao_grp_controle'],
                'distancia_euclidiana_indisp': row['distancia_euclidiana_indisp'],
                'distancia_euclidiana_vol_atms': match['distancia_total_vol']
            })

lista_ag_proximas_vol_atms = pd.DataFrame(resultados_vol_atms)

# Cálculo volume de Transações
resultados_vol_transacoes = []

for agencia_ref in lista_ag_proximas_vol_atms['agencia_ref'].unique():
    df_ref = lista_ag_proximas_vol_atms[lista_ag_proximas_vol_atms['agencia_ref'] == agencia_ref]

    distancias_transacoes = []
    for ag_cand in df_ref['agencia_grp_controle'].unique():
        df_cand = df_ref[df_ref['agencia_grp_controle'] == ag_cand]
        if df_cand.empty:
            continue

        distancia = np.linalg.norm(df_cand['vol_transacao_ref'] - df_cand['vol_transacao_grp_controle'])
        distancias_transacoes.append({'agencia_grp_controle': ag_cand, 'distancia_total_trans': distancia})

    top1 = sorted(distancias_transacoes, key=lambda x: x['distancia_total_trans'])[:1]

    for match in top1:
        ag_cand = match['agencia_grp_controle']
        df_cand = df_ref[df_ref['agencia_grp_controle'] == ag_cand]

        for _, row in df_cand.iterrows():
            resultados_vol_transacoes.append({
                'agencia_ref': agencia_ref,
                'anomes_ref': row['anomes_ref'],
                'agencia_grp_controle': ag_cand,
                'indice_indisp_ref': row['indice_indisp_ref'],
                'indice_indisp_grp_controle': row['indice_indisp_grp_controle'],
                'vol_atms_ref': row['vol_atms_ref'],
                'vol_atms_grp_controle': row['vol_atms_grp_controle'],
                'vol_transacao_ref': row['vol_transacao_ref'],
                'vol_transacao_grp_controle': row['vol_transacao_grp_controle'],
                'distancia_euclidiana_indisp': row['distancia_euclidiana_indisp'],
                'distancia_euclidiana_vol_atms': row['distancia_euclidiana_vol_atms'],
                'distancia_euclidiana_vol_trans': match['distancia_total_trans']
            })

lista_ag_proximas_vol_trans = pd.DataFrame(resultados_vol_transacoes)
agencias_grupo_controle = lista_ag_proximas_vol_trans[['agencia_ref','agencia_grp_controle','anomes_ref']].rename(columns={
    'anomes_ref': 'anomes'
})
agencias_grupo_controle['anomes'] = agencias_grupo_controle['anomes'].astype(int)
agencias_grupo_controle.to_csv('test/agencias_grupo_controle_final.csv', index=False)
agencias_grupo_controle.head()

,agencia_ref,agencia_grp_controle,anomes
0,9995,9427,202506
